In [ ]:
! pip3 install bayesian_optimization
! pip3 install cython

In [ ]:
! sudo apt-get update
! sudo apt-get -y install build-essential

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import cython
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%load_ext Cython

In [3]:
#train_values_short = pd.read_csv('train_values_short.csv', index_col='building_id')
train_values_short_s = pd.read_csv('train_values_short_superestructures.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [4]:
random_state = 

In [5]:
%%cython
def hassanat1(double[:] x, double[:] y, double[:] factores):
    cdef int n = x.shape[0]
    cdef double res = 0
    cdef double d, minimo, maximo

    for i in range(n):
        minimo = min(x[i],y[i])
        maximo = max(x[i],y[i])
        if (minimo >=0):
            d =  1 - ( (1+minimo)/(1+maximo) )
        else:
            d =  1 - ( (1+minimo+abs(minimo))/(1+maximo+abs(minimo)) )
        res += d * factores[i]
    
    return res

In [6]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, min=0):
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for resultado in resultados_importantes:
        print("target: " , resultado['target'])
        for key,value in resultado['params'].items():
            print("\t\t", key, ':', value)
        print()

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train_values_short,
                                train_labels, test_size=0.03, random_state=random_state)

In [24]:
k = 22

start = time.time()
factores = np.array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, 7.76639614])\
    .astype('double')
knn = KNeighborsClassifier(weights="distance",n_neighbors=k,
                  metric=hassanat1, metric_params={'factores':factores})
knn.fit(x_train, y_train.values.ravel())
prediction = knn.predict(x_test)
end = time.time()
print(f1_score(y_test, prediction, average='micro'))
print(end-start)

0.6296296296296297
26.696475982666016


In [37]:
x_train, x_test, y_train, y_test = train_test_split(train_values_short_s,
                                train_labels, test_size=0.03, random_state=random_state)

In [38]:
def funcion_a_optimizar(superestructures):
    
    k = 22
    
    factores = np.array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, superestructures,
 superestructures, superestructures, superestructures,
 superestructures, superestructures, superestructures, superestructures,
superestructures, superestructures, superestructures])\
        .astype('double')
    knn = KNeighborsClassifier(weights="distance",n_neighbors=k,
                  metric=hassanat1, metric_params={'factores':factores})
    knn.fit(x_train, y_train.values.ravel())
    prediction = knn.predict(x_test)
    return f1_score(y_test, prediction, average='micro')

In [54]:
limites = { "superestructures":(0.5,1.5)}

In [69]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    random_state=random_state,
    bounds_transformer=SequentialDomainReductionTransformer(),
    verbose=5,
    pbounds=limites,
)

str_random_state = "%s" %random_state
load_logs(optimizer, "./anteriores_" + str_random_state  + ".json")
logger = JSONLogger(path="./logs_nuevos_" + str_random_state  + ".json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.probe({ "superestructures":0.7626109262828163}, lazy=True)

In [ ]:
optimizer.probe({ "superestructures":0.706036013}, lazy=True)

In [71]:
optimizer.maximize(
    init_points=,
    n_iter=,
)

In [97]:
mostrar(optimizer.res)

target:  0.7407407407407407
		 age : 0.1331076390865485
		 area : 0.3139710742344524
		 count_floors : 1.9195786742228238
		 foundation : 2.199668387926665
		 geo : 7.508261370013642
		 ground_floor : 1.0
		 height : 0.5186702671250013
		 legal_ownership : 3.3974077335766553
		 other_floor : 1.0
		 plan_configuration : 1.0
		 roof : 1.0
		 secondary_use : 0.12558762490941516
		 superestructure : 7.508261370013642

target:  0.7407407407407407
		 age : 0.1331076390865485
		 area : 0.3139710742344524
		 count_floors : 1.9195786742228238
		 foundation : 2.199668387926665
		 geo : 7.0
		 ground_floor : 1.0
		 height : 0.5186702671250013
		 legal_ownership : 2.0
		 other_floor : 1.0
		 plan_configuration : 1.0
		 roof : 1.0
		 secondary_use : 0.12558762490941516
		 superestructure : 7.508261370013642

target:  0.7037037037037037
		 age : 0.1331076390865485
		 area : 0.3139710742344524
		 count_floors : 1.9195786742228238
		 foundation : 2.199668387926665
		 geo : 7.508261370013642
		 ground_